In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%autosave 20

Autosaving every 20 seconds


In [23]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import zscore

In [3]:
df = pd.read_csv("customer_segmentation.csv", encoding="cp1252")
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12-01-2010 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12-01-2010 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12-01-2010 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12-01-2010 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12-01-2010 08:26,3.39,17850.0,United Kingdom


In [4]:
df.dtypes
print("-----------------")
df.info()



InvoiceNo       object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
UnitPrice      float64
CustomerID     float64
Country         object
dtype: object

-----------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [5]:
df.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [6]:
df.shape[0]

541909

In [7]:
col = ['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate','UnitPrice', 'CustomerID', 'Country']
for i in col:
    print(f'{i} - {df[i].nunique()}')
    

InvoiceNo - 25900
StockCode - 4070
Description - 4223
Quantity - 722
InvoiceDate - 23260
UnitPrice - 1630
CustomerID - 4372
Country - 38


In [8]:
df['Country'].value_counts()

Country
United Kingdom          495478
Germany                   9495
France                    8557
EIRE                      8196
Spain                     2533
Netherlands               2371
Belgium                   2069
Switzerland               2002
Portugal                  1519
Australia                 1259
Norway                    1086
Italy                      803
Channel Islands            758
Finland                    695
Cyprus                     622
Sweden                     462
Unspecified                446
Austria                    401
Denmark                    389
Japan                      358
Poland                     341
Israel                     297
USA                        291
Hong Kong                  288
Singapore                  229
Iceland                    182
Canada                     151
Greece                     146
Malta                      127
United Arab Emirates        68
European Community          61
RSA                         58


In [9]:
df['InvoiceNo'].nunique()/df.shape[0]

0.047794002314041656

In [10]:
df.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [11]:
 df.describe(include = 'object')

,InvoiceNo,StockCode,Description,InvoiceDate,Country
count,541909,541909,540455,541909,541909
unique,25900,4070,4223,23260,38
top,573585,85123A,WHITE HANGING HEART T-LIGHT HOLDER,10/31/2011 14:41,United Kingdom
freq,1114,2313,2369,1114,495478


In [12]:
print(df.duplicated().sum())

5268


## Data Cleaning
Checking Null values

In [13]:
#customerId cannot be null as it is used to calculate the lifetime value
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [14]:
#So drop all the null values from CustomerID
df.dropna(inplace=True)

In [15]:
df.shape

(406829, 8)

In [18]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'],format='mixed')

In [24]:
z_scores = pd.DataFrame(zscore(df[df.select_dtypes(include='number').columns.tolist()]), columns = df.select_dtypes(include = 'number').columns.tolist())
z_scores

,Quantity,UnitPrice,CustomerID
0,-0.024373,-0.013135,1.495280
1,-0.024373,-0.001017,1.495280
2,-0.016331,-0.010250,1.495280
3,-0.024373,-0.001017,1.495280
4,-0.024373,-0.001017,1.495280
...,...,...,...
406824,-0.000247,-0.037661,-1.521763
406825,-0.024373,-0.019627,-1.521763
406826,-0.032415,0.009948,-1.521763
406827,-0.032415,0.009948,-1.521763


In [25]:
outliers = z_scores[(np.abs(z_scores) > 3).any(axis=1)]
outliers

,Quantity,UnitPrice,CustomerID
3102,-37.685254,-0.049491,0.321143
3645,7.285843,-0.041989,0.006600
3740,11.532041,-0.047327,0.855690
3741,5.580930,-0.034631,0.855690
3784,-0.044478,4.206005,-1.250404
...,...,...,...
405998,5.983033,-0.039536,1.553053
405999,4.776726,-0.039536,1.553053
406349,325.634086,-0.019916,0.675952
406350,-325.731083,-0.019916,0.675952
